In [1]:
import os, os.path
import numpy as np
from time import time
import pandas as pd
import math as m
import setup_runs as sr
import importlib
import matplotlib.pyplot as plt
import itertools

In [4]:
df_out = pd.read_csv(sr.fp_csv_output_multi_sector)
df_mas = pd.read_csv(sr.fp_csv_attribute_master)
df_fut = pd.read_csv(sr.fp_csv_attribute_future)
df_pr = pd.read_csv(sr.fp_csv_parameter_ranges)
df_pmr = pd.read_csv(sr.fp_csv_output_multi_sector_pmr.replace(".csv", "_tornado.csv"))
df_pmr_orig = pd.read_csv(sr.fp_csv_output_multi_sector_pmr.replace(".csv", "_orig.csv"))
#get ids not run by pmr
mas_nopmr = set(df_out["master_id"]) - set(df_pmr["master_id"])
#get df of values
df_mas_nopmr = df_mas[df_mas["master_id"].isin(mas_nopmr)][["master_id", "time_series_id", "future_id"]].copy()
df_mas_nopmr_fut0 = df_mas_nopmr[df_mas_nopmr["future_id"] == 0][["master_id", "time_series_id"]].copy().rename(columns = {"master_id": "master_id_0"})
df_mas_nopmr = pd.merge(df_mas_nopmr, df_mas_nopmr_fut0, how = "left", on = ["time_series_id"])

dict_ts_to_tornado_master = sr.build_dict(df_mas_nopmr_fut0[["time_series_id", "master_id_0"]])








In [5]:


#get baselines by time series
dict_ts_to_master = {0: 201, 1: 603}
#appendage
df_append = []
#loop over time series
for ts in list(dict_ts_to_master.keys()):
    #get key to extract from the PMR original dataset
    m_id_orig = int(dict_ts_to_master[ts])
    #subset
    df_sub = df_pmr_orig[df_pmr_orig["master_id"] == m_id_orig].copy()
    #get new master
    m_id_new = int(dict_ts_to_tornado_master[ts])
    #rename
    df_sub = df_sub.rename(columns = {"master_id": "master_id_0"})
    df_sub["master_id_0"] = df_sub["master_id_0"].replace({m_id_orig: m_id_new})
    #outer join
    df_new = df_mas_nopmr[df_mas_nopmr["master_id_0"] == m_id_new].copy()
    #outer
    df_new = pd.merge(df_new, df_sub, how = "outer", on = ["master_id_0"])
    
    df_append.append(df_new)

df_append = pd.concat(df_append, axis = 0)
fields_excl = [x for x in df_append.columns if ("_id" in x) and (x != "master_id")]
df_append = df_append[[x for x in df_append.columns if (x not in fields_excl)]]

#join
df_export = pd.concat([df_pmr, df_append], axis = 0).sort_values(by = ["master_id", "year"])



In [28]:
df_export

,master_id,year,el-emissions_total-mtco2e,el-costs_total_capex-UNITSHERE,el-costs_total_opex-UNITSHERE
0,239,2017,25.42,0.00,585.88
1,239,2018,27.53,0.00,641.49
2,239,2019,30.29,0.00,710.90
3,239,2020,33.49,0.00,941.31
4,239,2021,27.87,0.00,791.77
...,...,...,...,...,...
7441,955,2046,3.10,2822.56,1550.88
7442,955,2047,2.74,3150.63,1534.41
7443,955,2048,2.62,2675.04,1546.76
7444,955,2049,2.46,3159.04,1553.92


In [29]:
df_tmp = pd.merge(df_mas[df_mas["master_id"].isin(df_pmr["master_id"])][["master_id", "future_id"]].copy(), df_fut, how = "left", on = ["future_id"])
df_tmp = df_tmp[["master_id", "parameter", "range_value", "scale_value"]]

df_e = df_export[(df_export["master_id"].isin(df_pmr["master_id"])) & (df_export["year"] == 2050)].copy()
df_e = df_e[[x for x in df_e.columns if x != "year"]]

df_tmp = pd.merge(df_tmp, df_e, how = "left", on = ["master_id"])

df_tmp.to_csv("/Users/jsyme/Desktop/emissions_for_el.csv", index = None, encoding = "UTF-8")


In [30]:
df_tmp[df_tmp["el-emissions_total-mtco2e"] == 2.3]



,master_id,parameter,range_value,scale_value,el-emissions_total-mtco2e,el-costs_total_capex-UNITSHERE,el-costs_total_opex-UNITSHERE
2,406,trajmix_fuel_price_coal,max,1.0,2.3,3527.82,1592.57
3,407,trajmix_fuel_price_coal,min,0.0,2.3,3520.48,1590.69
4,408,trajmix_fuel_price_diesel,max,1.0,2.3,3527.82,1592.57
5,409,trajmix_fuel_price_diesel,min,0.0,2.3,3579.66,1592.37
6,410,trajmix_fuel_price_fuel_oil,max,1.0,2.3,3527.82,1592.57
7,411,trajmix_fuel_price_fuel_oil,min,0.0,2.3,3552.35,1597.03
8,412,trajmix_fuel_price_natural_gas,max,1.0,2.3,3527.82,1592.57
11,417,trajmix_investment_cost_csp_solar,min,0.0,2.3,3510.06,1590.96
12,418,trajmix_investment_cost_geothermal,max,10000.0,2.3,3579.27,1592.72
13,419,trajmix_investment_cost_geothermal,min,0.0,2.3,3528.18,1592.50


In [21]:
futs = list(set(df_mas[df_mas["master_id"].isin(df_pmr["master_id"])]["future_id"]))
futs.sort()
df_fut[df_fut["future_id"].isin(futs)];

df_export[df_export["year"]]
              

,master_id,year,el-emissions_total-mtco2e,el-costs_total_capex-UNITSHERE,el-costs_total_opex-UNITSHERE
0,239,2017,25.42,0.00,585.88
1,239,2018,27.53,0.00,641.49
2,239,2019,30.29,0.00,710.90
3,239,2020,33.49,0.00,941.31
4,239,2021,27.87,0.00,791.77
...,...,...,...,...,...
7441,955,2046,3.10,2822.56,1550.88
7442,955,2047,2.74,3150.63,1534.41
7443,955,2048,2.62,2675.04,1546.76
7444,955,2049,2.46,3159.04,1553.92


In [80]:
df_export.to_csv(sr.fp_csv_output_multi_sector_pmr, index = None, encoding = "UTF-8")

In [79]:
df_pmr.to_csv(sr.fp_csv_output_multi_sector_pmr.replace(".csv", "_tornado.csv"), index = None, encoding = "UTF-8")

In [7]:
df_export[(df_export["master_id"].isin(df_pmr["master_id"])) & (df_export["year"] == 2050)]

,master_id,year,el-emissions_total-mtco2e,el-costs_total_capex-UNITSHERE,el-costs_total_opex-UNITSHERE
33,256,2050,4.00,3527.82,1980.84
101,257,2050,1.27,3527.82,1381.47
169,406,2050,2.30,3527.82,1592.57
237,407,2050,2.30,3520.48,1590.69
305,408,2050,2.30,3527.82,1592.57
373,409,2050,2.30,3579.66,1592.37
441,410,2050,2.30,3527.82,1592.57
509,411,2050,2.30,3552.35,1597.03
577,412,2050,2.30,3527.82,1592.57
645,413,2050,4.85,2870.81,1756.15


In [78]:
df_pmr.shape

(1360, 5)

In [8]:
#from inventory (just top three gasses)
dict_vals = {
    "energy": [84121, 75.5, 3.8],
    "transportation": [26231.2, 5.3, 1.9],
    "energy_industry": [35483.7, 1.6, 0.6],
    "manufacturing_and_construction": [15684.7, 6.7, 0.9],
    "ippu": [3322.5 + 2869.1 + 272.3, 0.9, 1.5],
    "agriculture": [445.4, 263.6, 16],
    "forestry": [-65703, 5.1, 0.3],
    "circular_economy": [0.5, 217.5, 1.2]
}

vec_weight = np.array([1, 21, 310])

for k in dict_vals.keys():w
    
    val = round(np.dot(np.array(dict_vals[k]), vec_weight)/1000, 2)
    print("CO2E val for " + str(k) + " is: " + str(val))
    
    print("\n")

CO2E val for energy is: 86.88


CO2E val for transportation is: 26.93


CO2E val for energy_industry is: 35.7


CO2E val for manufacturing_and_construction is: 16.1


CO2E val for ippu is: 6.95


CO2E val for agriculture is: 10.94


CO2E val for forestry is: -65.5


CO2E val for circular_economy is: 4.94




In [3]:
np.dot(np.array(range(3)), np.array(range(5, 8)))

20

In [9]:
86.88 + 6.95 + 4.94 + 10.94 - 65.5

44.209999999999994

In [13]:
np.dot(np.array([22186.4, 562.6, 22.8]), vec_weight) + 2869.1 + 272.3

44210.4